In [1]:
import numpy as np  # Vectors and matrices
from scipy import stats  # Probability distributions
import matplotlib.pyplot as plt  # Plots
from matplotlib import style
import matplotlib as mpl
try:
    mpl.style.use('seaborn-v0_8')
except:
    mpl.style.use('seaborn')
mpl.rcParams['image.cmap'] = 'plasma'
nice_hist = dict(bins='auto', density=True, ec='w')

 # Exercice 2 : Algorithme de Metropolis <a id="part2"></a>
 
 **On veut simuler (approximativement) une loi normale centrée réduite grâce à l'algorithme de Metropolis.**

 
 
 > **Question 1.**
 On choisit comme noyau de proposition $q$ de sorte que, pour tout $x \in \mathbb R$, $q(x, \cdot)$ corresponde à la densité de la variable aléatoire $Y=x+U$ avec $U \sim \mathcal U_{[-1/2,1/2]}$.
 Autrement dit, $q(x, \cdot)$ est la densité de probabilité d'aller en $y$ sachant qu'on part du point $x$. Déterminer $q(x,y)$. En déduire $q(y,x)$.

 >**Question 2.**
    Traduire l'algorithme de Metropolis-Hastings dans ce cadre.


 >**Question 3.**
 Écrire une fonction `sample_metropolis(kernel, size=1, init=0.)` qui construit une chaîne de Markov de taille $n$ partant du point `init` selon l'algorithme décrit ci-avant.
 L'argument `kernel` représente la loi de $U$, ici : `kernel=stats.uniform(loc=-0.5, scale=1)`.
 La fonction devra retourner la réalisation de la chaîne et le taux d'acceptation.
 >

Appliquer cet algorithme pour construire une chaîne de longueur $n=10^4$ et partant d'un point $X_0$ tiré uniformément dans $[-3,3]$.



In [ ]:
n = 10**4

def sample_metropolis(kernel, size=1, init=0.):
    unif = stats.uniform()
    threshold = unif.rvs(size=size)  # Array of the n thresholds U_i
    move = kernel.rvs(size=size)  # Array of the n moves from the current position
    
    sample = [init]
    n_acc = 0
    for i in range(size):
        # Compléter

        # Fin compléter
    return np.asarray(sample), n_acc/size

# Answer


# Draw a sample with Metropolis
# Compléter

# Fin compléter


# >**Question 4.**
# Pour la réalisation de cette chaîne, représenter un histogramme des $X_i$ auquel on superposera la densité de la loi normale centrée réduite.
# Indiquer le taux d'acceptation dans le titre de la figure.
# >Que pouvez-vous en dire ?

# In[ ]:


# Answer
norm = stats.norm()

x = np.linspace(-3, 3)
# Compléter

# Fin compléter
plt.title(f"Taux d'acceptation : {rate}");


# **Réponse :**
# …

# >**Question 5.**
# Observer ce qu'il se passe lorsque la condition initiale est $X_0=10$.

# In[ ]:


# Answer


# **Réponse :**
# …

# >**Question 6.**
# Revenons à $X_0\sim{\cal U}_{[-3,3]}$. Plutôt qu'une loi de proposition uniforme sur $[-1/2,1/2]$, considérer une loi uniforme sur $[-10,10]$. Qu'observez-vous ? Et avec une loi uniforme sur $[-0.1,0.1]$ ?

# In[ ]:


# Answer


# **Réponse :**
# …

# # Exercice 3 : Metropolis vs rejet <a id="part3"></a>
# >On considère la loi $G$ de densité sur $\mathbb{R}^2$ proportionnelle à 
# $$ f(u,v) =   (\cos u ) ^2     ( \sin v) ^2  \operatorname{e}^{-0.05   (u^2 + v^2)}.$$
# >
# >**Question 1.**
# Définir une fonction `F(U, V)` retournant $f(U, V)$ et utiliser le code ci-dessous pour afficher une représentation 3D de $f$ sur $[-5,5]\times[-5,5]$.

# In[11]:


def plot_surf(U, V, Z, figsize=(15, 12)):
    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(projection='3d'))
    # ax.view_init(30, 120)
    plt.rcParams['axes.grid'] = False
    surf = ax.plot_surface(U, V, Z, rstride=1, cstride=1, cmap="coolwarm", linewidth=0)
    ax.contourf(U, V, Z, zdir='z', offset=Z.max()+1, cmap="coolwarm", alpha=0.5)
    ax.set_zlim(top=Z.max()+1)
    fig.colorbar(surf)
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    ax.set_zlabel('z')
    plt.rcParams['axes.grid'] = True
    return fig, ax


# In[ ]:


# Answer
def F(U, V):
    # Compléter

    # Fin compléter

u = np.linspace(-5, 5, num=100)
v = np.linspace(-5, 5, num=100)
U, V = np.meshgrid(u, v)  # All possible pairs from u and v

# Compléter

# Fin compléter
plot_surf(U, V, Z);


# **Réponse :**
# …

# >**Question 2.**
# On veut simuler (approximativement) selon la loi $G$ en utilisant l'algorithme de Metropolis-Hastings.
# >Partant de $x=(u,v)$, on considère le noyau de proposition $q$ défini de sorte que $q(x,\cdot)$ corresponde à la densité de $Y=x+\sigma W$, où $\sigma>0$ est un paramètre de réglage et $W\sim{\cal N} \left(\binom 00,I_2 \right)$, loi gaussienne centrée réduite dans $\mathbb{R}^2$.
# >Expliquer pourquoi nous sommes dans le cadre de l'algorithme de Metropolis.

# **Réponse :**
# …

# >**Question 3.**
# Implémenter l'algorithme de Metropolis. On utilisera le noyau de transition $q$ ci-dessus avec $\sigma=1$. On pourra par exemple prendre comme initialisation $X_1=(0,0)$ et considérer une chaîne $(X_k)_{0\leq k\leq n}$ de longueur $n=10^4$. En fin de simulation, afficher le taux global d'acceptation sur l'ensemble des mutations proposées.

# In[ ]:


# Answer
def f(x):
    u, v = x
    return F(u, v)

def sample_metropolis_2d(kernel, size=1, init=(0., 0.)):
    unif = stats.uniform()
    threshold = unif.rvs(size=size-1)  # Array of the n-1 thresholds U_i
    move = kernel.rvs(size=size-1)  # Array of the n-1 moves from the current position
    
    sample = [init]
    n_acc = 0
    for i in range(size-1):
        # Compléter

        # Fin compléter
    
    return np.asarray(sample), n_acc/size

n = 10**4
sigma = 1

# Compléter

# Fin compléter
print(f"Taux d'acceptation : {rate}")


# **Réponse :**
# …

# >**Question 4.**
# Sur le graphe de $f$ par lignes de niveau (`plt.contourf(U, V, Z, cmap=cm.coolwarm, alpha=0.5)`), superposer les points de la chaîne. Faire la même chose avec $\sigma$ grand, par exemple $\sigma=10$, et commenter. Idem avec $\sigma$ petit, par exemple $\sigma=0.1$. Afficher les taux d'acceptation dans les deux cas.

# In[ ]:


# Answer
plt.figure(figsize=(15, 15))
for i, sigma in enumerate([0.1, 1, 10]):
    # Compléter

    # Fin compléter
    plt.subplot(2, 2, i+1)
    plt.contourf(U, V, Z, cmap="coolwarm", alpha=0.5)
    # Compléter

    # Fin compléter
    plt.xlim([U.min(), U.max()])
    plt.ylim([V.min(), V.max()])
    plt.title(f"$\\sigma={sigma:.1f}$, taux d'acceptation : {rate:.2f}")


# **Réponse :**
# …

# >**Question 5.**
# Proposer une méthode de rejet pour simuler suivant la loi $G$ à partir d'une loi instrumentale gaussienne. Comme en question précédente, superposer un échantillon de grande taille simulé par cette méthode aux niveaux de la fonction $f$ pour vérifier visuellement le bon fonctionnement de l'algorithme.

# **Réponse :**
# …

# In[ ]:


# Answer
aux = stats.multivariate_normal(cov=10*np.eye(2))
unif = stats.uniform()

n = 10**3

sample = []
n_tot = 0
for i in range(n):
    candidate = aux.rvs()
    n_tot += 1
    # Compléter

    # Fin compléter
    sample.append(candidate)
sample = np.asarray(sample)
rate = n / n_tot

plt.figure(figsize=(8, 8))
plt.contourf(U, V, Z, cmap="coolwarm", alpha=0.5)
plt.scatter(*sample.T, c='r', s=1)
plt.xlim([U.min(), U.max()])
plt.ylim([V.min(), V.max()])
plt.title(f"Taux d'acceptation : {rate:.2f}");


# >**Question 6.**
# Des deux méthodes, laquelle vous semble préférable ?

# **Réponse :**
# …

# # Exercice 4 : Algorithme de Gibbs <a id="part4"></a>
# >Soit $(X,Y)$ un couple aléatoire de densité jointe sur $\mathbb R^2$, $f : (x,y) \mapsto e^{-y}\mathbf{1}_{0\leq x\leq y}$.
# >
# >**Question 1.**
# Déterminer la loi marginale de $X$.

# **Réponse :**
# …

# >**Question 2.**
# Sachant $X=x\geq0$, déterminer la densité conditionnelle de $Y | X=x$, notée $f_{Y | X=x}$. Quelle loi reconnaissez-vous ?

# **Réponse :**
# …

# >**Question 3.**
# En déduire une méthode pour simuler une réalisation du couple aléatoire $(X,Y)$. L'implémenter pour simuler un échantillon de couples $(X_1,Y_1),\dots,(X_n,Y_n)$ de taille $n=1000$. Représenter le nuage de points ainsi obtenu. 

# In[ ]:


# Answer
n = 1000

exp = stats.expon()

# Compléter

# Fin compléter

plt.figure(figsize=(8, 8))
plt.scatter(X, Y, s=1)


# >**Question 4.**
# Sachant $Y=y\geq 0$, déterminer la densité conditionnelle de $X | Y=y$, notée $f_{X | Y=y}$. Quelle loi reconnaissez-vous ?

# **Réponse :**
# …

# >**Question 5.**
# En partant par exemple du point $(x_0,y_0)=(0,1)$, proposer un échantillonneur de Gibbs pour obtenir une trajectoire $\left( (X_1, Y_1),\dots,(X_n, Y_n) \right)$ de densité cible $f$. Représenter le nuage de points ainsi obtenu.

# In[ ]:


# Answer
unif = stats.uniform()

X, Y = [0], [1]
for i in range(n-1):
    # Compléter

    # Fin compléter
    
plt.figure(figsize=(8, 8))
plt.scatter(X, Y, s=1)


# >**Question 6.**
# Des deux méthodes proposées, laquelle choisiriez-vous pour simuler selon la densité $f$ ?

# **Réponse :**
# …

# # Exercice 5 : Algorithme de Metropolis pour l'échantillonnage d'une loi a posteriori <a id="part5"></a>
# >On considère le modèle bayésien :
# \begin{cases}
#     \boldsymbol \theta \sim {\cal U}_{[0,1]} \\
#     (X_1, \dots, X_n) | \boldsymbol \theta \sim \left( \boldsymbol \theta \mathcal N(1, 1) + (1-\boldsymbol \theta) \mathcal N(-1, 1) \right)^{\otimes n}.
#     %\mathbf X | \boldsymbol \theta \sim \left( \boldsymbol \theta N(1, 1) + (1-\boldsymbol \theta) N(-1, 1) \right)^{\otimes n}.
# \end{cases}
# Notons $\phi$ la densité de $\mathcal N(0, 1)$. Ainsi, sachant $\boldsymbol \theta = \theta$,  $X_1$ a pour densité $\theta \phi(x-1) + (1-\theta) \phi(x+1)$.
# On remarque aussi que, sachant $\boldsymbol \theta = \theta$, $X_1\overset{\mathcal{L}}{=} Z Y_1 + (1-Z) Y_{-1}$, où $Y_1, Y_{-1}$ et $Z$ sont trois variables aléatoires indépendantes de lois respectives $\mathcal N(1, 1)$, $\mathcal N(-1, 1)$ et $\mathcal B(\theta)$.
# >
# >**Question 1.**
# Soit $\theta_0$ une réalisation de $\boldsymbol \theta$.
# Pour $n=100$, générer une réalisation de $X_1,\dots,X_n$ selon le mélange ci-dessus pondéré par $\theta_0$.

# In[ ]:


# Answer
prior = stats.uniform()
theta0 = prior.rvs()

ber = stats.bernoulli(theta0)
normp = stats.norm(loc=1)
normm = stats.norm(loc=-1)

n = 100
# Compléter

# Fin compléter

print(f'theta0 = {theta0}')
plt.hist(X, **nice_hist);


# >**Question 2.**
# Expliciter la loi a posteriori $\Pi[\cdot | \mathbf X]$ et l'estimateur de Bayes $\hat \theta_n$ pour la perte quadratique.

# **Réponse :**
# …

# >**Question 3.**
# En déduire un estimateur Monte-Carlo $\hat\theta_n^N$ de la réalisation de l'estimateur de Bayes $\hat\theta_n$. L'implémenter pour $N=500$ par exemple. 

# **Réponse :**
# …

# In[ ]:


# Answer
N = 500

# Compléter

# Fin compléter

plt.plot(np.arange(1, N+1), estMC, label = "Estimateur MC")
plt.axhline(theta0, color="red", label="$\\theta_0$")
plt.ylim(0, 1)
plt.legend();


# >**Question 4.**
# >On souhaite générer un échantillon suivant la loi a posteriori $\Pi[\cdot | \mathbf X]$. On adopte pour cela la méthode de Metropolis-Hastings (qui fournira donc approximativement un échantillon de loi $\Pi[\cdot | \mathbf X]$, i.e. une trajectoire de loi cible $\Pi[\cdot | \mathbf X]$) avec comme noyau de proposition $q$, de sorte que $q(\theta,\theta')=\mathbf 1_{[0,1]}(\theta')$.
# Quelle est la loi de proposition ?
# Que vaut le rapport de Metropolis-Hastings $r(\theta,\theta')$ ?
# >Avec la condition initiale $\boldsymbol \theta_1=1/2$, implémenter l'algorithme pour une chaîne de longueur $m=10^4$ et représenter un estimateur de la densité de la trajectoire $(\boldsymbol \theta_1,\dots,\boldsymbol \theta_m)$. Donner le taux global d'acceptation sur l'ensemble des mutations proposées.

# **Réponse :**
# …

# In[ ]:


# Answer
m = 10**4
init = 0.5

threshold = prior.rvs(size=m-1)
move = prior.rvs(size=m-1)

sample = [init]
for i in range(m-1):
    # Compléter

    # Fin compléter

plt.hist(sample, **nice_hist)
print(f"Taux d'acceptation : {len(np.unique(sample)) / m:.2f}")


# **Réponse :**
# …

# >**Question 5.**
# Mêmes questions en considérant le noyau de transition correspondant à $\boldsymbol \theta' = \boldsymbol \theta+U/\sqrt{n}$ (modulo 1), avec $U \sim \mathcal U_{[-1,1]}$.



# Exercice 6 : Slice sampler <a id="part6"></a>
>On considère la fonction $f : (u,x) \in \mathbb R^2 \mapsto \mathbf 1_{0<u<\frac{1}{2}\exp \left(-\sqrt{x} \right)}$.
>
>**Question 1.**
Montrer que $g : x \mapsto \frac{1}{2}\exp \left(-\sqrt{x} \right)\mathbf 1_{x>0}$ est une densité sur $\mathbb{R}$. En déduire que $f$ est une densité sur $\mathbb{R}^2$.



>**Question 2.**
Soit $(U, X)$ de densité $f$.
Déterminer, pour tout $(u, x) \in \mathbb R_+^2$, les lois conditionnelles de $U | X=x$ et $X | U=u$.


# >**Question 3.**
# En partant du point $(U_1,X_1)=(1/(4e),1)$, implémenter un échantillonneur de Gibbs pour obtenir une trajectoire $(U_1,X_1),\dots,(U_n,X_n)$ de taille $n=1000$ et de densité cible $f$.



>**Question 4.**
Sur un même graphe, représenter la densité $g$ et un estimateur de la densité obtenu à partir de la trajectoire $X_1,\dots,X_n$.